In [16]:
import cv2
import pywavefront
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull
from PIL import Image, ImageDraw
import io
import os

def generate_aruco_marker(id, size=500, dictionary=cv2.aruco.DICT_4X4_250):
    aruco_dict = cv2.aruco.Dictionary_get(dictionary)
    marker_img = cv2.aruco.drawMarker(aruco_dict, id, size)
    return marker_img

def plot_object(file_path):
    scene = pywavefront.Wavefront(file_path)
    items = list(scene.materials.items())
    ver = items[1][1].vertices
    n_ver = int(len(ver)/8)
    v = np.zeros((n_ver,3))
    for i in range(n_ver):
        v[i,:] = ver[8*i+5:8*i+5+3]
    v = v[np.argsort(v[:,2]),:]
    fig, ax = plt.subplots()
    
    ax.scatter(v[:3000,0],v[:3000,1])
    plt.xlabel('X')
    plt.ylabel('Y')
    ax.plot([0,0.01],[0,0],c='r')
    ax.plot([0,0],[0,0.01],c='g')
    ax.set_aspect('equal', 'box')

    hull = ConvexHull(v[:3000, :2])
    for simplex in hull.simplices:
        plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], 'k-')
    polygon = Polygon(hull.points[hull.vertices], fill=None, edgecolor='r', linewidth=1)
    ax.add_patch(polygon)
    
    points = hull.points[hull.vertices]
    x1, y1 = points[0]
    x2, y2 = points[1]
    m = (y2 - y1) / (x2 - x1)
    angle_with_x_axis = np.degrees(np.arctan(m))
    angle_with_y_axis = 90 - abs(angle_with_x_axis)
    ax.annotate(f"Angle with x-axis (degrees): {angle_with_x_axis:.2f}", xy=(1.05, 0.1), xycoords='axes fraction', ha='right')
    ax.annotate(f"Angle with y-axis (degrees): {angle_with_y_axis:.2f}", xy=(1.05, 0.05), xycoords='axes fraction', ha='right')

    x_ticks = np.arange(min(v[:,0]), max(v[:,0])+0.02, 0.02)
    y_ticks = np.arange(min(v[:,1]), max(v[:,1])+0.02, 0.02)
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    return fig, v

def get_intersection_point():
    return 0, 0

if __name__ == "__main__":
    marker_id = 6
    marker_img = generate_aruco_marker(marker_id)
    marker_pil_img = Image.fromarray(marker_img)
    
    fig, v = plot_object('/home/jose/Downloads/GraspBenchmarkWorkspace/YCB/models/ycb/006_mustard_bottle/tsdf/textured.obj')
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=300)
    buf.seek(0)
    plot_pil_img = Image.open(buf)
    
    a4_pixel_width = int(8.27 * 300)
    a4_pixel_height = int(11.69 * 300)
    a4_img = Image.new('RGB', (a4_pixel_width, a4_pixel_height), (255, 255, 255))

    marker_x_pos = (a4_pixel_width - marker_pil_img.width) // 2 - plot_pil_img.width // 6 - 18
    marker_y_pos = 0
    plot_x_pos = (a4_pixel_width - plot_pil_img.width) // 2
    plot_y_pos = a4_pixel_height - plot_pil_img.height - 520

    a4_img.paste(marker_pil_img, (marker_x_pos, marker_y_pos))
    a4_img.paste(plot_pil_img, (plot_x_pos, plot_y_pos))
    
    intersection_x, intersection_y = get_intersection_point()
    y_scale_factor = plot_pil_img.height / (max(v[:3000,1]) - min(v[:3000,1]))
    intersection_pixel_y = (max(v[:3000, 1]) - intersection_y) * y_scale_factor

    intersection_x_a4 = plot_x_pos + plot_pil_img.width / 2 - 18
    intersection_y_a4 = plot_y_pos + intersection_pixel_y + 1300

    draw = ImageDraw.Draw(a4_img)
    line_color = (0, 0, 0)
    draw.line([(intersection_x_a4, intersection_y_a4), 
               (intersection_x_a4, marker_y_pos + marker_pil_img.height)], fill=line_color)
    
    a4_img.save('/home/jose/Downloads/Final_Output.pdf', "PDF", resolution=100.0)
    plt.close()


In [29]:
import cv2
import pywavefront
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull
from PIL import Image, ImageDraw, ImageFont
import io

def generate_aruco_marker(id, size=500, dictionary=cv2.aruco.DICT_4X4_250):
    aruco_dict = cv2.aruco.Dictionary_get(dictionary)
    marker_img = cv2.aruco.drawMarker(aruco_dict, id, size)
    return marker_img

def plot_object(file_path):
    scene = pywavefront.Wavefront(file_path)
    items = list(scene.materials.items())
    ver = items[1][1].vertices
    n_ver = int(len(ver)/8)
    v = np.zeros((n_ver,3))
    for i in range(n_ver):
        v[i,:] = ver[8*i+5:8*i+5+3]
    v = v[np.argsort(v[:,2]),:]
    fig, ax = plt.subplots()
    
    ax.scatter(v[:3000,0],v[:3000,1])
    plt.xlabel('X')
    plt.ylabel('Y')
    ax.plot([0,0.01],[0,0],c='r')
    ax.plot([0,0],[0,0.01],c='g')
    ax.set_aspect('equal', 'box')

    hull = ConvexHull(v[:3000, :2])
    for simplex in hull.simplices:
        plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], 'k-')
    polygon = Polygon(hull.points[hull.vertices], fill=None, edgecolor='r', linewidth=1)
    ax.add_patch(polygon)
    
    points = hull.points[hull.vertices]
    x1, y1 = points[0]
    x2, y2 = points[1]
    m = (y2 - y1) / (x2 - x1)
    angle_with_x_axis = np.degrees(np.arctan(m))
    angle_with_y_axis = 90 - abs(angle_with_x_axis)
    ax.annotate(f"Angle with x-axis (degrees): {angle_with_x_axis:.2f}", xy=(1.05, 0.1), xycoords='axes fraction', ha='right')
    ax.annotate(f"Angle with y-axis (degrees): {angle_with_y_axis:.2f}", xy=(1.05, 0.05), xycoords='axes fraction', ha='right')

    x_ticks = np.arange(min(v[:,0]), max(v[:,0])+0.02, 0.02)
    y_ticks = np.arange(min(v[:,1]), max(v[:,1])+0.02, 0.02)
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    return fig, v

def draw_axes_on_a4(a4_img, v, width, height):
    draw = ImageDraw.Draw(a4_img)
    try:
        font = ImageFont.truetype("arial.ttf", size=12)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()  # If Arial isn't available, use the default PIL font

    # X-axis
    draw.line([(0, height - 1), (width, height - 1)], fill=(0, 0, 0))
    # Y-axis
    draw.line([(0, 0), (0, height)], fill=(0, 0, 0))
    
    x_min, x_max = min(v[:, 0]), max(v[:, 0])
    y_min, y_max = min(v[:, 1]), max(v[:, 1])

    x_range = x_max - x_min
    y_range = y_max - y_min

    x_ticks_interval = width // 20
    y_ticks_interval = height // 20

    x_value_interval = x_range / 20
    y_value_interval = y_range / 20

    # X-ticks and labels
    for i in range(0, width + 1, x_ticks_interval):
        draw.line([(i, height - 1), (i, height - 20)], fill=(0, 0, 0))
        value = x_min + (i / width) * x_range
        draw.text((i - 10, height - 40), f"{value:.2f}", font=font, fill=(0, 0, 0))

    # Y-ticks and labels
    for i in range(0, height + 1, y_ticks_interval):
        draw.line([(0, i), (20, i)], fill=(0, 0, 0))
        value = y_max - (i / height) * y_range
        draw.text((25, i - 10), f"{value:.2f}", font=font, fill=(0, 0, 0))

    return a4_img

def get_intersection_point():
    return 0, 0

if __name__ == "__main__":
    marker_id = 6
    marker_img = generate_aruco_marker(marker_id)
    marker_pil_img = Image.fromarray(marker_img)
    
    fig, v = plot_object('/home/jose/Downloads/GraspBenchmarkWorkspace/YCB/models/ycb/006_mustard_bottle/tsdf/textured.obj')
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=300)
    buf.seek(0)
    plot_pil_img = Image.open(buf)
    
    a4_pixel_width = int(8.27 * 300)
    a4_pixel_height = int(11.69 * 300)
    a4_img = Image.new('RGB', (a4_pixel_width, a4_pixel_height), (255, 255, 255))

    marker_x_pos = (a4_pixel_width - marker_pil_img.width) // 2 - plot_pil_img.width // 6 - 18
    marker_y_pos = 0
    plot_x_pos = (a4_pixel_width - plot_pil_img.width) // 2
    plot_y_pos = a4_pixel_height - plot_pil_img.height - 520

    a4_img.paste(marker_pil_img, (marker_x_pos, marker_y_pos))
    a4_img.paste(plot_pil_img, (plot_x_pos, plot_y_pos))
    
    # Draw the axes on the A4 image
    a4_img = draw_axes_on_a4(a4_img, v, a4_pixel_width, a4_pixel_height)

    intersection_x, intersection_y = get_intersection_point()
    y_scale_factor = plot_pil_img.height / (max(v[:3000,1]) - min(v[:3000,1]))
    intersection_pixel_y = (max(v[:3000, 1]) - intersection_y) * y_scale_factor

    intersection_x_a4 = plot_x_pos + plot_pil_img.width / 2 - 18
    intersection_y_a4 = plot_y_pos + intersection_pixel_y + 1300

    draw = ImageDraw.Draw(a4_img)
    line_color = (0, 0, 0)
    draw.line([(intersection_x_a4, intersection_y_a4), 
               (intersection_x_a4, marker_y_pos + marker_pil_img.height)], fill=line_color)
    
    a4_img.save('/home/jose/Downloads/Final_Output.pdf', "PDF", resolution=100.0)
    plt.close()


In [33]:
import cv2
import pywavefront
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull
from PIL import Image, ImageDraw, ImageFont
import io

def generate_aruco_marker(id, size=500, dictionary=cv2.aruco.DICT_4X4_250):
    aruco_dict = cv2.aruco.Dictionary_get(dictionary)
    marker_img = cv2.aruco.drawMarker(aruco_dict, id, size)
    return marker_img

def plot_object(file_path):
    scene = pywavefront.Wavefront(file_path)
    items = list(scene.materials.items())
    ver = items[1][1].vertices
    n_ver = int(len(ver)/8)
    v = np.zeros((n_ver,3))
    for i in range(n_ver):
        v[i,:] = ver[8*i+5:8*i+5+3]
    v = v[np.argsort(v[:,2]),:]
    min_z = np.min(v[:3000, 2])  # Added this line for z-axis minimum value
    max_z = np.max(v[:3000, 2])  # Added this line for z-axis maximum value
    fig, ax = plt.subplots()
    
    
    ax.scatter(v[:3000,0],v[:3000,1])
    plt.xlabel('X')
    plt.ylabel('Y')
    ax.plot([0,0.01],[0,0],c='r')
    ax.plot([0,0],[0,0.01],c='g')
    ax.set_aspect('equal', 'box')

    hull = ConvexHull(v[:3000, :2])
    for simplex in hull.simplices:
        plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], 'k-')
    polygon = Polygon(hull.points[hull.vertices], fill=None, edgecolor='r', linewidth=1)
    ax.add_patch(polygon)
    
    points = hull.points[hull.vertices]
    x1, y1 = points[0]
    x2, y2 = points[1]
    m = (y2 - y1) / (x2 - x1)
    angle_with_x_axis = np.degrees(np.arctan(m))
    angle_with_y_axis = 90 - abs(angle_with_x_axis)
    ax.annotate(f"Angle with x-axis (degrees): {angle_with_x_axis:.2f}", xy=(1.05, 0.1), xycoords='axes fraction', ha='right')
    ax.annotate(f"Angle with y-axis (degrees): {angle_with_y_axis:.2f}", xy=(1.05, 0.05), xycoords='axes fraction', ha='right')
    ax.annotate(f"Z Min: {min_z:.6f}", xy=(1.05, 0.25), xycoords='axes fraction', ha='right')
    ax.annotate(f"Z Max: {max_z:.6f}", xy=(1.05, 0.20), xycoords='axes fraction', ha='right')


    x_ticks = np.arange(min(v[:,0]), max(v[:,0])+0.02, 0.02)
    y_ticks = np.arange(min(v[:,1]), max(v[:,1])+0.02, 0.02)
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    return fig, v


def draw_axes_on_a4(a4_img, v, width, height):
    draw = ImageDraw.Draw(a4_img)
    try:
        font = ImageFont.truetype("arial.ttf", size=12)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()  # If Arial isn't available, use the default PIL font

    draw.line([(0, height - 1), (width, height - 1)], fill=(0, 0, 0))  # X-axis
    draw.line([(0, 0), (0, height)], fill=(0, 0, 0))  # Y-axis
    
    x_min, x_max = min(v[:, 0]), max(v[:, 0])
    y_min, y_max = min(v[:, 1]), max(v[:, 1])

    x_ticks_interval = width // 20
    y_ticks_interval = height // 20

    x_range = 0.210
    y_range = 0.297

    x_value_interval = x_range / 20
    y_value_interval = y_range / 20

    x_values_drawn = set()
    y_values_drawn = set()
    
    draw.text((1400, height - 3000), "X-Axis", font=font, fill=(0, 0, 0))
    draw.text((1700, 200), "Y-Axis", font=font, fill=(0, 0, 0))
    


    for i in range(0, width + 1, x_ticks_interval):
        value = x_min-0.07 + (i / width) * x_range
        value_rounded = round(value, 2)
        if value_rounded not in x_values_drawn:
            x_values_drawn.add(value_rounded)
            draw.line([(i, height - 1), (i, height - 20)], fill=(0, 0, 0))
            draw.text((i - 10, height - 40), f"{value_rounded:.2f}", font=font, fill=(0, 0, 0))

    for i in range(0, height + 1, y_ticks_interval):
        value = y_max+0.15 - (i / height) * y_range
        value_rounded = round(value, 2)
        if value_rounded not in y_values_drawn:
            y_values_drawn.add(value_rounded)
            draw.line([(0, i), (20, i)], fill=(0, 0, 0))
            draw.text((25, i - 10), f"{value_rounded:.2f}", font=font, fill=(0, 0, 0))

    return a4_img

def get_intersection_point():
    return 0, 0

if __name__ == "__main__":
    marker_id = 6
    marker_img = generate_aruco_marker(marker_id)
    marker_pil_img = Image.fromarray(marker_img)
    marker_pil_img = marker_pil_img.rotate(-90, expand=1)
    file_path ="/home/jose/Downloads/GraspBenchmarkWorkspace/YCB/models/ycb/061_foam_brick/tsdf/textured.obj"
    # file_path = '/home/jose/Downloads/GraspBenchmarkWorkspace/YCB/models/ycb/006_mustard_bottle/tsdf/textured.obj'
    object_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))  

    fig, v = plot_object(file_path)

    
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=310)
    buf.seek(0)
    plot_pil_img = Image.open(buf)
    
    a4_pixel_width = int(8.27 * 300)
    a4_pixel_height = int(11.69 * 300)
    a4_img = Image.new('RGB', (a4_pixel_width, a4_pixel_height), (255, 255, 255))

    marker_x_pos = (a4_pixel_width - marker_pil_img.width) // 2 - plot_pil_img.width // 6 +610
    marker_y_pos = 0
    plot_x_pos = (a4_pixel_width - plot_pil_img.width) // 2 - 34
    plot_y_pos = a4_pixel_height - plot_pil_img.height - 960 

    a4_img.paste(marker_pil_img, (marker_x_pos, marker_y_pos))
    a4_img.paste(plot_pil_img, (plot_x_pos, plot_y_pos))

    # Adding the title to the A4 image
    draw = ImageDraw.Draw(a4_img)
    try:
        title_font = ImageFont.truetype("arial.ttf", size=40)  # Adjust the size as needed
    except OSError:
        title_font = ImageFont.load_default()
    draw.text((50, 50), object_name, font=title_font, fill=(0, 0, 0))  # Adjust the position (50, 50) as needed
    
    
    # Draw the axes on the A4 image
    a4_img = draw_axes_on_a4(a4_img, v, a4_pixel_width, a4_pixel_height)

    intersection_x, intersection_y = get_intersection_point()
    y_scale_factor = plot_pil_img.height / (max(v[:3000,1]) - min(v[:3000,1]))
    intersection_pixel_y = (max(v[:3000, 1]) - intersection_y) * y_scale_factor

    intersection_x_a4 = plot_x_pos + plot_pil_img.width / 2 + 138
    intersection_y_a4 = plot_y_pos + intersection_pixel_y + 1300

    draw = ImageDraw.Draw(a4_img)
    line_color = (0, 0, 0)
    try:
        font = ImageFont.truetype("arial.ttf", size=30)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()
    draw.line([(intersection_x_a4, intersection_y_a4), 
               (intersection_x_a4, marker_y_pos + marker_pil_img.height)], fill=line_color)
    draw.text((intersection_x_a4 + 10, (intersection_y_a4 + marker_y_pos + marker_pil_img.height) / 2), 
          "10 cm", font=font, fill=line_color)
    
    output_pdf_path = f'/home/jose/Downloads/{object_name}.pdf'
    a4_img.save(output_pdf_path, "PDF", resolution=100.0)
    plt.close()

In [ ]:
import cv2
import pywavefront
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull
from PIL import Image, ImageDraw, ImageFont
import io

def generate_aruco_marker(id, size=500, dictionary=cv2.aruco.DICT_4X4_250):
    aruco_dict = cv2.aruco.Dictionary_get(dictionary)
    marker_img = cv2.aruco.drawMarker(aruco_dict, id, size)
    return marker_img

def plot_object(file_path):
    scene = pywavefront.Wavefront(file_path)
    items = list(scene.materials.items())
    ver = items[1][1].vertices
    n_ver = int(len(ver)/8)
    v = np.zeros((n_ver,3))
    for i in range(n_ver):
        v[i,:] = ver[8*i+5:8*i+5+3]
    v = v[np.argsort(v[:,2]),:]
    min_z = np.min(v[:3000, 2])  # Added this line for z-axis minimum value
    max_z = np.max(v[:3000, 2])  # Added this line for z-axis maximum value
    fig, ax = plt.subplots()
    
    
    ax.scatter(v[:3000,0],v[:3000,1])
    plt.xlabel('X')
    plt.ylabel('Y')
    ax.plot([0,0.01],[0,0],c='r')
    ax.plot([0,0],[0,0.01],c='g')
    ax.set_aspect('equal', 'box')

    hull = ConvexHull(v[:3000, :2])
    for simplex in hull.simplices:
        plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], 'k-')
    polygon = Polygon(hull.points[hull.vertices], fill=None, edgecolor='r', linewidth=1)
    ax.add_patch(polygon)
    
    points = hull.points[hull.vertices]
    x1, y1 = points[0]
    x2, y2 = points[1]
    m = (y2 - y1) / (x2 - x1)
    angle_with_x_axis = np.degrees(np.arctan(m))
    angle_with_y_axis = 90 - abs(angle_with_x_axis)
    ax.annotate(f"Angle with x-axis (degrees): {angle_with_x_axis:.2f}", xy=(1.05, 0.1), xycoords='axes fraction', ha='right')
    ax.annotate(f"Angle with y-axis (degrees): {angle_with_y_axis:.2f}", xy=(1.05, 0.05), xycoords='axes fraction', ha='right')
    ax.annotate(f"Z Min: {min_z:.6f}", xy=(1.05, 0.25), xycoords='axes fraction', ha='right')
    ax.annotate(f"Z Max: {max_z:.6f}", xy=(1.05, 0.20), xycoords='axes fraction', ha='right')


    x_ticks = np.arange(min(v[:,0]), max(v[:,0])+0.02, 0.02)
    y_ticks = np.arange(min(v[:,1]), max(v[:,1])+0.02, 0.02)
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    return fig, v
def draw_axes_on_a4(a4_img, v, width, height):
    draw = ImageDraw.Draw(a4_img)
    try:
        font = ImageFont.truetype("arial.ttf", size=12)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()  # If Arial isn't available, use the default PIL font

    draw.line([(0, height - 1), (width, height - 1)], fill=(0, 0, 0))  # X-axis
    draw.line([(0, 0), (0, height)], fill=(0, 0, 0))  # Y-axis
    
    x_min, x_max = min(v[:, 0]), max(v[:, 0])
    y_min, y_max = min(v[:, 1]), max(v[:, 1])

    x_ticks_interval = width // 20
    y_ticks_interval = height // 20

    x_range = 0.210
    y_range = 0.297

    x_value_interval = x_range / 20
    y_value_interval = y_range / 20

    x_values_drawn = set()
    y_values_drawn = set()
    
    draw.text((1400, height - 3000), "X-Axis", font=font, fill=(0, 0, 0))
    draw.text((1700, 200), "Y-Axis", font=font, fill=(0, 0, 0))
    


    for i in range(0, width + 1, x_ticks_interval):
        value = x_min-0.07 + (i / width) * x_range
        value_rounded = round(value, 2)
        if value_rounded not in x_values_drawn:
            x_values_drawn.add(value_rounded)
            draw.line([(i, height - 1), (i, height - 20)], fill=(0, 0, 0))
            draw.text((i - 10, height - 40), f"{value_rounded:.2f}", font=font, fill=(0, 0, 0))

    for i in range(0, height + 1, y_ticks_interval):
        value = y_max+0.15 - (i / height) * y_range
        value_rounded = round(value, 2)
        if value_rounded not in y_values_drawn:
            y_values_drawn.add(value_rounded)
            draw.line([(0, i), (20, i)], fill=(0, 0, 0))
            draw.text((25, i - 10), f"{value_rounded:.2f}", font=font, fill=(0, 0, 0))

    return a4_img

def get_intersection_point():
    return 0, 0

if __name__ == "__main__":
    marker_id = 6
    marker_img = generate_aruco_marker(marker_id)
    marker_pil_img = Image.fromarray(marker_img)
    marker_pil_img = marker_pil_img.rotate(-90, expand=1)
    file_path = '/home/jose/Downloads/GraspBenchmarkWorkspace/YCB/models/ycb/006_mustard_bottle/tsdf/textured.obj'
    object_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))  

    fig, v = plot_object(file_path)

    
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=310)
    buf.seek(0)
    plot_pil_img = Image.open(buf)
    
    a4_pixel_width = int(8.27 * 300)
    a4_pixel_height = int(11.69 * 300)
    a4_img = Image.new('RGB', (a4_pixel_width, a4_pixel_height), (255, 255, 255))

    marker_x_pos = (a4_pixel_width - marker_pil_img.width) // 2 - plot_pil_img.width // 6 +454
    marker_y_pos = 0
    plot_x_pos = (a4_pixel_width - plot_pil_img.width) // 2 - 34
    plot_y_pos = a4_pixel_height - plot_pil_img.height - 960 

    a4_img.paste(marker_pil_img, (marker_x_pos, marker_y_pos))
    a4_img.paste(plot_pil_img, (plot_x_pos, plot_y_pos))

    # Adding the title to the A4 image
    draw = ImageDraw.Draw(a4_img)
    try:
        title_font = ImageFont.truetype("arial.ttf", size=40)  # Adjust the size as needed
    except OSError:
        title_font = ImageFont.load_default()
    draw.text((50, 50), object_name, font=title_font, fill=(0, 0, 0))  # Adjust the position (50, 50) as needed
    
    
    # Draw the axes on the A4 image
    a4_img = draw_axes_on_a4(a4_img, v, a4_pixel_width, a4_pixel_height)

    intersection_x, intersection_y = get_intersection_point()
    y_scale_factor = plot_pil_img.height / (max(v[:3000,1]) - min(v[:3000,1]))
    intersection_pixel_y = (max(v[:3000, 1]) - intersection_y) * y_scale_factor

    intersection_x_a4 = plot_x_pos + plot_pil_img.width / 2 - 18
    intersection_y_a4 = plot_y_pos + intersection_pixel_y + 1300

    draw = ImageDraw.Draw(a4_img)
    line_color = (0, 0, 0)
    try:
        font = ImageFont.truetype("arial.ttf", size=30)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()
    draw.line([(intersection_x_a4, intersection_y_a4), 
               (intersection_x_a4, marker_y_pos + marker_pil_img.height)], fill=line_color)
    draw.text((intersection_x_a4 + 10, (intersection_y_a4 + marker_y_pos + marker_pil_img.height) / 2), 
          "10 cm", font=font, fill=line_color)
    
    output_pdf_path = f'/home/jose/Downloads/{object_name}.pdf'
    a4_img.save(output_pdf_path, "PDF", resolution=100.0)
    plt.close()